<a href="https://colab.research.google.com/github/MathewNhari/Sendy-Logistics-Challenge/blob/master/Sendy_Logistics_Challenge_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Libaries**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Import Data

In [ ]:

riders = pd.read_csv('Riders.csv')
SampleSubmission = pd.read_csv('SampleSubmission.csv')
Test_df = pd.read_csv('Test.csv')
Train_df = pd.read_csv('Train.csv', parse_dates= ['Placement - Day of Month'])
VariableDefinitions = pd.read_csv('VariableDefinitions.csv')

**Variable Definations**

In [ ]:
VariableDefinitions

,Order No,Unique number identifying the order
0,User Id,Unique number identifying the customer on a pl...
1,Vehicle Type,"For this competition limited to bikes, however..."
2,Platform Type,"Platform used to place the order, there are 4 ..."
3,Personal or Business,Customer type
4,Placement - Day of Month,Placement - Day of Month i.e 1-31
5,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
6,Placement - Time,Placement - Time - Time of day the order was p...
7,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
8,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)
9,Confirmation - Time,Confirmation - Time - Time of day the order wa...


**Sample Submission**

In [ ]:
SampleSubmission.head()

,Order_No,Time from Pickup to Arrival
0,Order_No_19248,567.0
1,Order_No_12736,4903.0
2,Order_No_768,5649.0
3,Order_No_15332,NaN
4,Order_No_21373,NaN


**Data**

In [ ]:
riders.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


In [ ]:
Test_df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


Joining the Test_df and Train_df to riders 

In [ ]:
Test = Test_df.merge(riders,on='Rider Id',how='inner')
Train = Train_df.merge(riders,on='Rider Id',how='inner')

In [ ]:
Train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,3:41:17 PM,18,5,3:41:30 PM,18,5,4:07:16 PM,18,5,4:13:37 PM,18,5,5:01:43 PM,20,24.5,NaN,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,12:51:41 PM,31,5,1:12:49 PM,31,5,1:20:02 PM,31,5,1:33:08 PM,31,5,2:16:43 PM,6,24.7,NaN,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,7:12:10 AM,2,2,7:12:29 AM,2,2,7:35:24 AM,2,2,7:38:46 AM,2,2,8:28:32 AM,18,15.2,NaN,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,10:40:58 AM,22,2,10:42:24 AM,22,2,10:56:00 AM,22,2,10:59:40 AM,22,2,11:26:22 AM,7,19.2,NaN,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [ ]:
#if no output is returned, the columns are equal otherwise returns 'Not True' as Assertion Error.
#input the column names manually
#checking if months columns are equal then checking the weekdays
assert Train['Pickup - Day of Month'].all() == Train['Arrival at Destination - Day of Month'].all(), "Not True"

In [ ]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21201 entries, 0 to 21200
Data columns (total 33 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  object 
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

# **Cleaning Data**


Checking for Duplicates

In [ ]:
#No duplicate Data On Test Data
def Check_duplicates(df):
  duplicates = df.duplicated(subset = ['Order No'], keep = False)
  return df[duplicates]

Checking Duplicates on Train Data

In [ ]:
#No duplicates 
Train_duplicates = Check_duplicates(Train)
Test_duplucates = Check_duplicates(Test)

Drop Columns

In [ ]:
def columnDrop(df,columns):
  df.drop(columns, axis=1,inplace=True)
  df.set_index(['Order No'], inplace= True)
  return df

In [ ]:
Drop_Train_Cols = ['Vehicle Type', 'User Id', 'Rider Id', 'Precipitation in millimeters','Arrival at Destination - Time']

Drop_Test_Cols = ['Vehicle Type', 'User Id', 'Rider Id','Precipitation in millimeters']
       
Test= columnDrop(Test,Drop_Test_Cols)
Train = columnDrop(Train,Drop_Train_Cols)

**Changing  into The Correct DataType**

In [ ]:
 category_cols = ['Platform Type', 'Pickup - Weekday (Mo = 1)', 'Personal or Business']
       
Train[category_cols] = Train[category_cols].astype('category')
Test[category_cols] = Test[category_cols].astype('category')


In [ ]:
  def timeconvert(df,Time):
    df[Time] = df[Time].astype('datetime64')
    return df

In [ ]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train = timeconvert(Train,Time_columns)
Test = timeconvert(Test,Time_columns)

Setting Time columns into datetime Data type


In [ ]:
def SplitDatetime(df,columns):
  for item in columns:
    df[item+'_day'] = pd.to_datetime(df[item]).dt.day
    df[item+'_hour'] = pd.to_datetime(df[item]).dt.hour
    df[item+'_minutes'] = pd.to_datetime(df[item]).dt.minute
    df[item+'_seconds'] = pd.to_datetime(df[item]).dt.second
  df.drop(columns,axis=1, inplace = True)
  return df

In [ ]:
Time_columns = ['Placement - Time', 'Confirmation - Time',
                'Arrival at Pickup - Time','Pickup - Time']
Train = SplitDatetime(Train,Time_columns)
Test = SplitDatetime(Test,Time_columns)

In [ ]:
Train.head(2)

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_day,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_day,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_day,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_day,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,3,Business,9,5,9,5,9,5,9,5,9,5,4,20.4,-1.317755,36.830370,-1.300406,36.829741,745,1637,1309,13.8,549,1,9,35,46,1,9,40,10,1,10,4,47,1,10,27,30
Order_No_27440,3,Personal,18,5,18,5,18,5,18,5,18,5,20,24.5,-1.326774,36.787807,-1.356237,36.904295,2886,1637,1309,13.8,549,1,15,41,17,1,15,41,30,1,16,7,16,1,16,13,37


In [ ]:
Test.head()

,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_day,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_day,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_day,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_day,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_19248,3,Business,27,3,27,3,27,3,27,3,8,NaN,-1.333275,36.870815,-1.305249,36.822390,439,1511,13.3,171,1,16,44,10,1,16,44,29,1,16,53,4,1,17,6,47
Order_No_2699,1,Personal,26,2,26,2,26,2,26,2,19,NaN,-1.312745,36.875715,-1.288435,36.760677,439,1511,13.3,171,1,14,43,21,1,14,47,16,1,15,9,26,1,15,17,48
Order_No_21486,3,Business,29,5,29,5,29,5,29,5,9,19.7,-1.255189,36.782203,-1.293393,36.790237,439,1511,13.3,171,1,11,55,5,1,11,58,12,1,12,25,49,1,12,28,54
Order_No_19336,3,Business,2,1,2,1,2,1,2,1,15,16.4,-1.310451,36.868724,-1.259542,36.787118,439,1511,13.3,171,1,9,23,21,1,9,23,44,1,9,44,50,1,10,5,31
Order_No_20374,3,Business,14,4,14,4,14,4,14,4,15,21.3,-1.333275,36.870815,-1.255189,36.782203,439,1511,13.3,171,1,12,55,59,1,13,22,44,1,13,49,38,1,13,51,49


**Taking Care of Missing Data**

In [ ]:
from sklearn.impute import SimpleImputer

def missingData(df,columns):
  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
  imputer.fit(df[columns])
  df[columns] = imputer.transform(df[columns])
  return df

In [ ]:
columns = ['Temperature']
Test = missingData(Test,columns)
Train = missingData(Train,columns)

**Get dummy Variables**

In [ ]:
def dummy_encoder(df, cols):
  dummies = pd.get_dummies(df,columns= cols,drop_first = True)
  merged = pd.concat([df,dummies],axis='columns')
  final = merged.drop(cols, axis = 1)
  return dummies


In [ ]:
cols = ['Platform Type','Personal or Business', 'Pickup - Weekday (Mo = 1)']
Train = dummy_encoder(Train, cols)
Test = dummy_encoder(Test, cols)
Test.head(1)

,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_day,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_day,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_day,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_day,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_19248,27,3,27,3,27,3,27,8,23.24612,-1.333275,36.870815,-1.305249,36.82239,439,1511,13.3,171,1,16,44,10,1,16,44,29,1,16,53,4,1,17,6,47,0,1,0,0,0,1,0,0,0,0


In [ ]:
Train.head()

,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_day,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_day,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_day,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_day,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,9,5,9,5,9,5,9,9,5,4,20.4,-1.317755,36.830370,-1.300406,36.829741,745,1637,1309,13.8,549,1,9,35,46,1,9,40,10,1,10,4,47,1,10,27,30,0,1,0,0,0,0,0,1,0,0
Order_No_27440,18,5,18,5,18,5,18,18,5,20,24.5,-1.326774,36.787807,-1.356237,36.904295,2886,1637,1309,13.8,549,1,15,41,17,1,15,41,30,1,16,7,16,1,16,13,37,0,1,0,1,0,0,0,1,0,0
Order_No_14170,31,5,31,5,31,5,31,31,5,6,24.7,-1.255189,36.782203,-1.273412,36.818206,2615,1637,1309,13.8,549,1,12,51,41,1,13,12,49,1,13,20,2,1,13,33,8,0,1,0,0,0,0,0,1,0,0
Order_No_21727,2,2,2,2,2,2,2,2,2,18,15.2,-1.290315,36.757377,-1.223520,36.802061,2986,1637,1309,13.8,549,1,7,12,10,1,7,12,29,1,7,35,24,1,7,38,46,0,1,0,1,1,0,0,0,0,0
Order_No_8301,22,2,22,2,22,2,22,22,2,7,19.2,-1.273524,36.799220,-1.300431,36.752427,1602,1637,1309,13.8,549,1,10,40,58,1,10,42,24,1,10,56,0,1,10,59,40,1,0,0,1,1,0,0,0,0,0


#Preprocessing

**Getting our Training Data into Features and Response variables**

In [ ]:
def XyFeatures(df):
  X = df.drop(['Time from Pickup to Arrival'], axis=1)
  y = df[['Time from Pickup to Arrival']]
  return (X,y)

In [ ]:
X = XyFeatures(Train)[0]
y = XyFeatures(Train)[1]

Spliting Data into Training and Test Set

In [ ]:
# import train/test split module
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [ ]:
columns = ['Distance (KM)', 'Temperature', 'Pickup Lat','Pickup Long','Destination Lat','Destination Long','No_Of_Orders',
           'Age', 'Average_Rating', 'No_of_Ratings']
X[columns] = scaler_X.fit_transform(X[columns])
y = scaler_y.fit_transform(y)

In [ ]:
X.head(1)

,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Placement - Time_day,Placement - Time_hour,Placement - Time_minutes,Placement - Time_seconds,Confirmation - Time_day,Confirmation - Time_hour,Confirmation - Time_minutes,Confirmation - Time_seconds,Arrival at Pickup - Time_day,Arrival at Pickup - Time_hour,Arrival at Pickup - Time_minutes,Arrival at Pickup - Time_seconds,Pickup - Time_day,Pickup - Time_hour,Pickup - Time_minutes,Pickup - Time_seconds,Platform Type_2,Platform Type_3,Platform Type_4,Personal or Business_Personal,Pickup - Weekday (Mo = 1)_2,Pickup - Weekday (Mo = 1)_3,Pickup - Weekday (Mo = 1)_4,Pickup - Weekday (Mo = 1)_5,Pickup - Weekday (Mo = 1)_6,Pickup - Weekday (Mo = 1)_7
Order No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Order_No_4211,9,5,9,5,9,5,9,9,5,-0.97137,-0.887322,-1.189424,0.509897,-0.511888,0.414159,-0.035206,0.501451,-0.090082,0.516144,1,9,35,46,1,9,40,10,1,10,4,47,1,10,27,30,0,1,0,0,0,0,0,1,0,0


In [ ]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.20,
                                                    random_state=1)

In [ ]:
def subset(df):
  X_subset = df.drop(['Placement - Time_day', 'Confirmation - Time_day','Arrival at Pickup - Time_day',
                    'Pickup - Time_day'], axis=1)
  return X_subset 

In [ ]:

X_train_subset = subset(X)
X_test_subset = subset(Test)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train_subset,
                                                        y,
                                                        test_size=0.20,
                                                        random_state=1,
                                                        shuffle=False)

# Feature Scaling

In [ ]:
##columns = ['Distance (KM)', 'Temperature', 'Pickup Lat','Pickup Long','Destination Lat','Destination Long','No_Of_Orders',
##           'Age', 'Average_Rating', 'No_of_Ratings']
# save standardized features into new variable
##X_train[columns] = scaler_X.fit_transform(X_train[columns])
#y_train = scaler_y.fit_transform(y_train)
##X_test[columns] = scaler_X.fit_transform(X_test[columns])
#y_test = scaler_y.fit_transform(y_test)

In [ ]:
##X_train2[columns] = scaler_X.fit_transform(X_train2[columns])
#y_train2 = scaler_y.fit_transform(y_train2)
##X_test2[columns] = scaler_X.fit_transform(X_test2[columns])
#y_test2 = scaler_y.fit_transform(y_test2)

# Building SVR model

**Traing the Lasso Regression model on the Training Set**

In [ ]:
from sklearn.svm import SVR
sv_reg = SVR(kernel='rbf', gamma='auto')
sv_reg.fit(X_train,y_train)
sub_sv_reg = SVR(kernel='rbf', gamma='auto')
sub_sv_reg.fit(X_train2,y_train2)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_pred = scaler_y.inverse_transform(sv_reg.predict(X_test))
sub_y_pred = scaler_y.inverse_transform(sub_sv_reg.predict(X_test2))

In [ ]:
from sklearn.metrics import mean_squared_error
SVR_rmse = mean_squared_error(y_test, y_pred, squared= False)
sub_SVR_rmse = mean_squared_error(y_test2, sub_y_pred, squared= False)

In [ ]:
SVR_rmse

1465.2063911764437

In [ ]:
sub_SVR_rmse

1443.5614650396785